# Eodag basics
____________

We present `eodag`'s basic features for **searching and downloading EO products**.

To be able to follow this tutorial, you will need to install the additional Python package [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html).

## Configuration

Let's start by setting your personal credentials to access [PEPS service](https://peps.cnes.fr) by filling your username and password below:

In [1]:
import os
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Let's check that `ipyleaflet` is available:

In [2]:
import ipyleaflet as ipyl

Then we create a workspace directory `eodag_workspace_basics` where all our files and configuration will live:

In [3]:
workspace = 'eodag_workspace_basics'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

Next, we create a configuration YAML file based on the following minimal content and save it in the workspace folder as `eodag_conf.yml`. Note that if we don't create such a configuration file `eodag` will load a default configuration from `~/.config/eodag/eodag.yml`.

```yaml
peps:
    download:
        outputs_prefix: "absolute_path_to_eodag_workspace"
        extract: true
```

This configuration file will indicate `eodag` to search for products made available by the PEPS service only, to download them in our workspace folder and to extract the downloaded products.

The credentials could also have been defined in the configuration file. Since we have set them with environment variables earlier, we are good to go.

In [4]:
yaml_content = """
peps:
    download:
        outputs_prefix: "{}"
        extract: true
""".format(os.path.abspath(workspace))
with open(os.path.join(workspace, 'eodag_conf.yml'), "w") as f_yml:
    f_yml.write(yaml_content.strip())

## Search

The first step before performing a **search** is to initialize the session by creating an `EODataAccessGateway` instance with the previous configuration file.

In [5]:
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
from eodag.utils.logging import setup_logging
setup_logging(verbose=1)

from eodag.api.core import EODataAccessGateway

dag = EODataAccessGateway(os.path.join(workspace, 'eodag_conf.yml'))

2021-01-11 20:51:38,107-15s eodag.config                     [INFO    ] Loading user configuration from: /home/maxime/TRAVAIL/06_EODAG/eodag/examples/eodag_workspace_basics/eodag_conf.yml
2021-01-11 20:51:38,316-15s eodag.core                       [INFO    ] Locations configuration loaded from /home/maxime/.config/eodag/locations.yml


Now let's search for Sentinel 2 L1C products over a period of 6 months in 2018 in the South of France, the search area being defined by its bounding box.

In [6]:
from datetime import date

product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': 0,
    'lonmax': 2,
    'latmin': 43,
    'latmax': 45
}

products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start='2018-06-01',
    end='2018-12-01',
    geom=extent
)

2021-01-11 20:51:38,523-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: peps
2021-01-11 20:51:38,524-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-12-01&geometry=POLYGON ((0.0000 43.0000, 0.0000 45.0000, 2.0000 45.0000, 2.0000 43.0000, 0.0000 43.0000))&productType=S2MSI1C&maxRecords=1&page=1
2021-01-11 20:51:38,947-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-12-01&geometry=POLYGON ((0.0000 43.0000, 0.0000 45.0000, 2.0000 45.0000, 2.0000 43.0000, 0.0000 43.0000))&productType=S2MSI1C&maxRecords=20&page=1
2021-01-11 20:51:39,818-15s eodag.core                       [INFO    ] Found 767 result(s) on provider 'peps'


In [7]:
estimated_total_nbr_of_results

767

In [8]:
len(products)

20

In [9]:
products[:3]

[EOProduct(id=S2A_MSIL1C_20181130T110411_N0207_R094_T30TYR_20181130T113958, provider=peps), EOProduct(id=S2A_MSIL1C_20181130T110411_N0207_R094_T31TCJ_20181130T113958, provider=peps), EOProduct(id=S2A_MSIL1C_20181130T110411_N0207_R094_T30TYQ_20181130T113958, provider=peps)]

`eodag` sent requests to the PEPS service and got as a result an estimated number of 767 (you might get a different value) products available matching the search criteria. The search also returned 20 products stored in the `products` variable that is a `SearchResult` instance, a list-like object. Each one of the products is an `EOProduct` instance that contains all the information required to download the product later.

You can see that the number of retrieved products is lower than the estimated number of products available. This is the same behaviour as when you make a search on Google where you get 10 results per page. `eodag` works the same way, it is set by default to return the results from *page 1*, a single page being defined by a collection of *20 products*. This behaviour can be adapted with the parameters `page` and `items_per_page` of the `search` method.

Now that we have found some products, we can easily store them in a GeoJSON file that we may use later to download them:

In [10]:
search_result_file = dag.serialize(products, filename=os.path.join(workspace, 'search_results.geojson'))

The products can also be converted into a Python dictionnary structured as GeoJSON file. Doing so allows us to pass this dictionnary to an interactive plotting library such as `ipyleaflet` to check the extents of the products before downloading them:

In [11]:
m = ipyl.Map(center=[43.6, 1.5], zoom=5)
layer = ipyl.GeoJSON(data=products.as_geojson_object(), hover_style={'fillColor': 'yellow'})

m.add_layer(layer)
m

Map(center=[43.6, 1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## Download

Finally we could easily **download** all the returned products with `dag.download_all(products)`. Since products are heavy (a single product may already be over 700 Mo) we show how to download a single product with `dag.download(product)`. We also demonstrate the use of the `NotebookProgressCallback` to add a nice download progress bar:

In [12]:
from eodag.utils import NotebookProgressCallback

# Download the second product found.
path = dag.download(products[1], progress_callback=NotebookProgressCallback())
path

2021-01-11 20:51:39,950-15s eodag.plugins.download.base      [INFO    ] Download url: https://peps.cnes.fr/resto/collections/S2ST/849bf704-d60b-5263-af6d-493784eb5cff/download


2021-01-11 20:51:42,173-15s eodag.plugins.download.base      [INFO    ] Extraction activated

Extracting files from S2A_MSIL1C_20181130T110411_N0207_R094_T31TCJ_20181130T113958.zip: 100%|██████████| 115/115 [00:00<00:00, 1282.60file/s]
2021-01-11 20:51:42,273-15s eodag.api.product                [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://peps.cnes.fr/resto/collections/S2ST/849bf704-d60b-5263-af6d-493784eb5cff/download


'file:///home/maxime/TRAVAIL/06_EODAG/eodag/examples/eodag_workspace_basics/S2A_MSIL1C_20181130T110411_N0207_R094_T31TCJ_20181130T113958/S2A_MSIL1C_20181130T110411_N0207_R094_T31TCJ_20181130T113958.SAFE'

When the download has completed you should find the product in your workspace folder. Note that above we set up the download with the Python object (`SearchResult`) returned by the search. It is also possible to recreate this search ouput thanks to the GeoJSON file we have saved previously with `dag.deserialize(path_to_geojson)`.

This concludes the tutorial for the basic usage of `eodag`.